In [3]:
import pandas as pd
import numpy as np
import os
import glob
import pgeocode
import geopandas as gpd

# Raw Dataset
raw_df = pd.read_csv('../dataset/annualy.csv')

# GEOID -> Location
gdf = gpd.read_file("../dataset/tl_2024_us_county.shp")

## 설정

In [5]:
BASE_PATH = os.path.join(os.path.curdir, '../results')
BASE_PATH = os.path.abspath(BASE_PATH)

BASE_PATH


'/Volumes/KUUWANGE/WORKSPACE/P_Personal/SOYBEAN/results'

## 전처리

In [8]:
### 재정렬 / 컬럼명 정리

df = raw_df.rename(columns={
    'COMMODITY_DESC': 'crop',
    'YEAR': 'year',
    'STATE_NAME': 'state',
    "COUNTRY_NAME": "country",
    'COUNTY_NAME': 'county',
    'DOMAIN_DESC': 'domain',
    'STATISTICCAT_DESC': 'type',
    'UNIT_DESC': 'unit',
    'VALUE': 'value',
    'LOCATION_DESC': 'loc',
    'COUNTRY_CODE': 'country_code',
    'STATE_FIPS_CODE': 'state_fips',
    'COUNTY_CODE': 'county_code'
    }, inplace=False).copy()
df = df[['crop', 'year', 'country',  'country_code',  'state', 'state_fips',  'county', 'county_code', 'type', 'domain', 'unit', 'value', 'loc']].copy()
df['country_code'] = df['country_code'].astype(int)
df['state_fips'] = df['state_fips'].astype(int)
df['county_code'] = df['county_code'].astype(int)
df = df.sort_values(by=['year', 'state', 'county', 'crop', 'domain', 'type'], ascending=[True, True, True, True, True, True], inplace=False).copy().reset_index(drop=True)

df["type_unit"] = df["type"] + " [" + df["unit"] + "]"
df_pivot = df.pivot_table(index=["year", "state", "state_fips", "county", "county_code", "crop"],
    columns="type_unit",
    values="value"
).reset_index()

df_pivot.columns.name = None

#raw_data = df.copy()

#df.assign(C=df['type'])

df_pivot = df_pivot[["year", "state", "state_fips", "county", "county_code", "crop", "AREA HARVESTED [ACRES]", "AREA PLANTED [ACRES]", "YIELD [BU / ACRE]", "PRODUCTION [BU]"]]
df_pivot = df_pivot.rename(columns={
    'AREA HARVESTED [ACRES]': 'area_harvested',
    'AREA PLANTED [ACRES]': 'area_planted',
    'YIELD [BU / ACRE]': 'yield',
    'PRODUCTION [BU]': 'production'
}).copy()

df = df_pivot.reset_index(drop=True)

raw_data = df.copy()

## 유틸 함수

In [9]:

### STATE_FIPS, COUNTY_FIPS 를 이용하여 LAT / LON, BOUNDARY 가져옴 ( CDMWF 사용 할 용도)
def find_boundary(state_fips, county_fips):
    filtered_gdf = gdf[(gdf['STATEFP'] == str(state_fips).zfill(2)) & (gdf['COUNTYFP'] == str(county_fips).zfill(3))]
    target_row = None
    if not filtered_gdf.empty:
        target_row = filtered_gdf.iloc[0]['geometry']
    else:
        return None

    if target_row is not None:
        (minx, miny, maxx, maxy) = target_row.bounds
        print(f"Bounds: {minx}, {miny}, {maxx}, {maxy}")
        return target_row

x = find_boundary(41, 1)
print(x.bounds)
# bound to lat lon
(minlat, minlon, maxlat, maxlon) = x.bounds
print(f"Bounds: {minlat}, {minlon}, {maxlat}, {maxlon}")

Bounds: -118.519477, 44.25509, -116.78371, 45.080746
(-118.519477, 44.25509, -116.78371, 45.080746)
Bounds: -118.519477, 44.25509, -116.78371, 45.080746


## Training

### 설정

In [10]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [11]:
## 데이터 정리 및 정규화, 데이터셋 생성

df = raw_data.copy()

## 데이터프레임중, 2000년 이후의 데이터만 남기기 ( NDVI )

df = df[df['year'] >= 2000].copy().reset_index(drop=True)


df['area_harvested'] = df['area_harvested'].replace(np.nan, 0)
df['yield'] = df['yield'].replace(np.nan, 0)
df['production'] = df['production'].replace(np.nan, 0)
df['area_planted'] = df['area_planted'].replace(np.nan, 0)

df['time_idx']  = df['year'] - df['year'].min()
df['group'] = df['crop'] + "_" + df['state'] + "_" + df['county']
df['group'] = df['group'].astype("category")
df['time_idx'] = df['time_idx'].astype("int64")

# drop na

df = df.dropna(subset=['area_harvested', 'area_planted', 'yield', 'production'], how='any').copy()
df = df.dropna(subset=['state', 'county', 'crop'], how='any').copy()
df = df.dropna(subset=['time_idx', 'group'], how='any').copy()
df = df.dropna(subset=['year'], how='any').copy()
df = df.dropna(subset=['state_fips'], how='any').copy()

df = df[['time_idx', 'group', 'year', 'state', 'county', 'crop', 'area_harvested', 'area_planted', 'yield', 'production']].copy()

sizes = df.groupby(['group']).size()

# drop groups with less than 14 rows
df = df[df['group'].isin(sizes[sizes >= 14].index)].copy()

/var/folders/4r/ccmn80ps0ms033hdrcww86kc0000gn/T/ipykernel_71547/2560259120.py:30: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sizes = df.groupby(['group']).size()


In [12]:
df = df[(df['group'].isin(['CORN_FLORIDA_BROWARD']) == False)]

In [13]:
max_prediction_length = 3
max_encoder_length = 6
training_cutoff = df["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],  # use all data up to the cutoff
    #data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="production",
    group_ids=["group"],
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["state", "crop", "county"],
    static_reals=[],
    # time_varying_known_reals=[col for col in df.columns if "ndvi_" in col or "prec_" in col or "temp_" in col],
    time_varying_known_reals=["area_harvested", "year"],
    time_varying_unknown_reals=["yield"],
    #variable_groups={"special_days": special_days},  # group of categorical variables can be treated as one variable
    #time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"],
    time_varying_unknown_categoricals=[],
    # time_varying_unknown_reals=[
    #     "volume",
    #     "log_volume",
    #     "industry_volume",
    #     "soda_volume",
    #     "avg_max_temp",
    #     "avg_volume_by_agency",
    #     "avg_volume_by_sku",
    # ],
    target_normalizer=GroupNormalizer(
        groups=["group"], transformation="softplus"
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
)

print (training)

# create validation set (predict=True) which means to predict the last max_prediction_length points in time
# for each series
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=False)

# create dataloaders for model
batch_size = 64  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=1, persistent_workers=True)

val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=1, persistent_workers=True)

TimeSeriesDataSet[length=108632](
	time_idx='time_idx',
	target='production',
	group_ids=['group'],
	weight=None,
	max_encoder_length=6,
	min_encoder_length=3,
	min_prediction_idx=np.int64(0),
	min_prediction_length=1,
	max_prediction_length=3,
	static_categoricals=['state', 'crop', 'county'],
	static_reals=[],
	time_varying_known_categoricals=None,
	time_varying_known_reals=['area_harvested', 'year'],
	time_varying_unknown_categoricals=[],
	time_varying_unknown_reals=['yield'],
	variable_groups=None,
	constant_fill_strategy=None,
	allow_missing_timesteps=True,
	lags=None,
	add_relative_time_idx=True,
	add_target_scales=True,
	add_encoder_length=True,
	target_normalizer=GroupNormalizer(
	method='standard',
	groups=['group'],
	center=True,
	scale_by_group=False,
	transformation='softplus',
	method_kwargs={}
),
	categorical_encoders={'__group_id__group': NaNLabelEncoder(add_nan=False, warn=True), 'group': NaNLabelEncoder(add_nan=False, warn=True), 'state': NaNLabelEncoder(add_nan=False, 

/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pytorch_forecasting/data/timeseries.py:1693: UserWarning: If predicting, no randomization should be possible - setting stop_randomization=True
  warnings.warn(
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pytorch_forecasting/data/timeseries.py:1831: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 1 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__group': 'CORN_IDAHO_OTHER (COMBINED) COUNTIES'}]
  warnings.warn(


### 학습

In [ ]:
early_stop_callback = EarlyStopping(monitor="train_loss", min_delta=1e-4, patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor(logging_interval="step", log_momentum=True)  # log learning rate and momentum
logger = TensorBoardLogger(save_dir=os.path.join(BASE_PATH, 'tf_logger'))  # logging results to a tensorboard

pl.seed_everything(42)

## (*•؎ •*) 은하수를 여행하는 히치하이커 (*•؎ •*) ##

trainer = pl.Trainer(
    max_epochs=50,
    # max_steps=50,
    #accelerator="cpu", # tpu / cuda
    # accelerator="cuda",
    # accelerator="cpu",
    # accelerator=,
    enable_model_summary=False,
    gradient_clip_val=0.1,
    limit_train_batches=100,
    # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    log_every_n_steps=2,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.01,
    hidden_size=42,
    attention_head_size=2,
    dropout=0.25,
    hidden_continuous_size=16, # 연속 변수의 표현 차원수
    loss=QuantileLoss(),
    log_interval=8,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    log_val_interval=10,  # uncomment for best val on last log_val_interval evaluations
    # optimizer="Ranger",
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Seed set to 42
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


Number of parameters in network: 170.7k


#### 실험

In [15]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    # check_val_every_n_epoch=1,
    #fast_dev_run=True,
    # gradient_clip_val=0.1,
    # limit_train_batches=100,
    # limit_val_batches=100,
)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric train_loss improved. New best score: 1041254.875


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric train_loss did not improve in the last 10 records. Best score: 547921.688. Signaling Trainer to stop.


### 최적화

In [ ]:
import pickle

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# 파라미터 튜닝

study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path=os.path.join(BASE_PATH, "TFT"),
    n_trials=1, # 몇번 시도할지
    max_epochs=2,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,
    verbose=2,
    # use Optuna to find ideal learning rate or use in-built learning rate finder
)

with open(os.path.join(BASE_PATH, "test_study.pkl"), "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

[I 2025-05-06 21:55:39,273] A new study created in memory with name: no-name-473e4c65-c478-4f51-9637-1badf1a86811
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pytorch_forecasting/models/temporal_fusion_transformer/tuning.py:142: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gradient_clip_val = trial.suggest_loguniform(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pytorch_forecasting/models/temporal_fusion_transformer/tuning.py:168: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout=trial.suggest_uniform("dropout", *dropout_range

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/Users/hacker/.pyenv/versions/3.11.12/lib/python3.11/site-packages/pytorch_forecasting/metrics/base_metrics.py:897: UserWarning: Loss is not finite. Resetting it to 1e9
  warnings.warn("Loss is not finite. Resetting it to 1e9")


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
[I 2025-05-06 21:57:41,392] Trial 0 finished with value: 1500066.375 and parameters: {'gradient_clip_val': 0.2954643463723628, 'hidden_size': 99, 'dropout': 0.21918646593882604, 'hidden_continuous_size': 34, 'attention_head_size': 2, 'learning_rate': 0.0015266674010786445}. Best is trial 0 with value: 1500066.375.


{'gradient_clip_val': 0.2954643463723628, 'hidden_size': 99, 'dropout': 0.21918646593882604, 'hidden_continuous_size': 34, 'attention_head_size': 2, 'learning_rate': 0.0015266674010786445}


### 훈련후 검증

In [ ]:
study

In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
# best_model_path = "/content/drive/MyDrive/crop/tf_logger/lightning_logs/version_6/checkpoints/epoch=10-step=110.ckpt"
print (best_model_path)


best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

# calcualte mean absolute error on validation set
predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))
MAE()(predictions.output, predictions.y)


IsADirectoryError: [Errno 21] Is a directory: '/Volumes/KUUWANGE/WORKSPACE/P_Personal/SOYBEAN/scripts'

In [ ]:
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True, return_index=True)

interpretation = best_tft.interpret_output(raw_predictions.output, reduction="sum")
best_tft.plot_interpretation(interpretation)


for idx in range(13):  # plot 10 examples
  plot_ind = best_tft.plot_prediction(raw_predictions.x, raw_predictions.output, idx=idx, add_loss_to_title=True, show_future_observed=True, plot_attention=False)
  idx_name = (raw_predictions.index['group'][idx])
  plot_ind.suptitle(f"Group: {idx_name}")
  # plot_ind.set(title=f"Group: {idx_name}")
  plot_ind.show()

In [ ]:
from pytorch_forecasting.metrics import SMAPE

# calcualte metric by which to display
#predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"))

#print (predictions.output)

#y_pred = predictions.y[0].cpu()



# 예측
predictions = best_tft.predict(val_dataloader, return_y=True, trainer_kwargs=dict(accelerator="cpu"), return_index=True)

# 예측값과 실제값 (둘 다 CPU로 옮겨서 계산)
y_pred = predictions.output.cpu()
y_true = predictions.y[0].cpu()

# SMAPE 인스턴스 생성
smape = SMAPE()

print (y_pred)
print (y_true)
# 전체 평균 SMAPE 계산
smape_score = smape(y_pred, y_true)

print(f"📊 SMAPE (전체 평균): {smape_score * 100:.2f}%")



# mean_losses = SMAPE(reduction="none")(predictions.output, predictions.y).mean(1)
# indices = mean_losses.argsort(descending=True)  # sort losses
# for idx in range(10):  # plot 10 examples
    # best_tft.plot_prediction(
    #     raw_predictions.x,
    #     raw_predictions.output,
    #     idx=indices[idx],
    #     add_loss_to_title=SMAPE(quantiles=best_tft.loss.quantiles),
    # )


predictions = best_tft.predict(val_dataloader, return_x=True)
predictions_vs_actuals = best_tft.calculate_prediction_actual_by_variable(predictions.x, predictions.output)
best_tft.plot_prediction_actual_by_variable(predictions_vs_actuals)